In [ ]:
!pip install pandas
!pip install pandas openpyxl
!pip install chardet

In [ ]:
!pip install hangul-romanize

In [ ]:
import pandas as pd

# 엑셀 파일을 읽기
df = pd.read_excel('MarriedNetwork.xlsx', sheet_name='MarriedNetwork_r 성종세대까지')

# 본인_id와 본인_관직가중치를 딕셔너리로 만들기
id_to_weight = dict(zip(df['본인_id'], df['본인_관직가중치']))

# 본인_조부_id와 본인_id가 같은 경우, 본인_관직가중치 값을 본인_조부_관직가중치에 복사
# df['본인_조부_관직가중치'] = df.apply(lambda row: id_to_weight.get(row['본인_조부_id'])
#                                   if row['본인_조부_id'] == row['본인_id']
#                                   else None, axis=1)
df['본인_조부_관직가중치'] = df['본인_조부_id'].map(id_to_weight)

# 결과를 새 엑셀 파일로 저장
df.to_excel('Updated_MarriedNetwork.xlsx', index=False)


In [ ]:
import pandas as pd
import json
from hangul_romanize.rule import academic
from hangul_romanize import Transliter

# 한글을 로마자로 바꾸는 변환기 초기화
transliter = Transliter(academic)

# 엑셀 파일을 열기
xls = pd.ExcelFile('MarriedNetwork.xlsx', engine='openpyxl')

# 특정 시트 데이터를 불러오기
df_married_network = xls.parse('MarriedNetwork_r 성종세대까지')

# NaN 값을 None으로 변경
df_married_network = df_married_network.where(pd.notnull(df_married_network), None)

# '변수번호' 칼럼의 값을 정수로 변환
df_married_network['변수번호'] = df_married_network['변수번호'].apply(lambda x: int(x) if pd.notna(x) else x)

# 0927: 배우자 부 데이터 항목 추가
categories = {
    '본인': [
        '본인_id', '본인_id2', 'string', '본인_성', '본인_명', '본인_성별', '본인_세대', '촌수', '본인_본관', '본인_봉호',
        '본인_묘호', '본인_관직', '본인_관직가중치',
        '본인_부_id', '본인_부_이름', '본인_부_관직가중치',
        '본인_조부_id', '본인_조부_이름', '본인_조부_관직가중치',
        '배우자_id', '배우자_성명', '배우자_성명2', '배우자_본관', '배우자_관직', '배우자_관직가중치',
        '배우자_부_id',	'배우자_부_이름',	'배우자_부_관직',	'배우자_부_관직가중치', '변수번호'
    ]
}

nodes = []
added_ids = set()  # 이미 추가된 '본인_id'를 추적

missing_columns = []
column_list = df_married_network.columns.tolist()
for category, columns in categories.items():
    for column in columns:
        if column not in column_list:
            missing_columns.append((category, column))

if missing_columns:
    for category, column in missing_columns:
        print(f"In category '{category}', '{column}' is missing from the provided column list.")
else:
    print("All columns in the categories dictionary exist in the provided column list.")

# 한 번의 반복문만 사용합니다.
for category, columns in categories.items():
    for _, row in df_married_network.iterrows():
        if row['본인_id'] not in added_ids:
            node = {"category": category}
            for column in columns:
                node[column] = row[column]

            # '본인_성'과 '본인_명'의 로마자 버전을 추가
            if node['본인_성'] is not None:
                node['본인_영문성'] = transliter.translit(node['본인_성'])
            else:
                node['본인_영문성'] = None

            if node['본인_명'] is not None:
                node['본인_영문명'] = transliter.translit(str(node['본인_명']))  # 변환을 위해 문자열로 캐스팅
            else:
                node['본인_영문명'] = None


            nodes.append(node)
            added_ids.add(row['본인_id'])

# JSON 형태로 변환
json_data = json.dumps(nodes, ensure_ascii=False, indent=4)

# JSON 형태로 파일에 저장
with open('nodes.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

In [ ]:
import json

# 노드 데이터 불러오기
with open('nodes.json', 'r', encoding='utf-8') as json_file:
    nodes = json.load(json_file)

# 엣지 데이터를 저장할 빈 리스트 생성
edges = []

# ID와 이름을 가져오는 함수
def get_id_and_name_from_node(node, category):
    id_keys = {
        '본인': '본인_id',
        '본인_부': '본인_부_id',
        '본인_조부': '본인_조부_id',
        '배우자': '배우자_id'
    }
    name_keys = {
        '본인': '본인_명',
        '본인_부': '본인_부_이름',
        '본인_조부': '본인_조부_이름',
        '배우자': '배우자_성명'
    }
    return node[id_keys[category]], node[name_keys[category]]

# 노드 ID를 key로 하는 딕셔너리 생성
node_dict = {node['본인_id']: node for node in nodes}

# 각 노드에 대해 참조 관계 확인 후 엣지 생성
for node in nodes:
    source_id = node['본인_id']
    source_name = node['본인_명']

    # 본인_부, 본인_조부, 배우자에 대한 참조 확인
    for relation in ['본인_부', '본인_조부', '배우자']:
        target_id, target_name = get_id_and_name_from_node(node, relation)

        # 참조 ID가 존재하고 해당 ID를 key로 하는 노드가 존재할 경우 엣지 생성
        if target_id and target_id in node_dict:
            # 항상 길이가 짧은 ID가 source가 되도록 정렬
            if len(source_id) > len(target_id):
                source_id, target_id = target_id, source_id
                source_name, target_name = target_name, source_name
                relation_str = f"{relation} - 본인"
            else:
                relation_str = f"본인 - {relation}"

            edge = {
                "source": source_id,
                "target": target_id,
                "source_name": source_name,
                "target_name": target_name,
                "relation": relation_str,
                "변수번호": node["변수번호"]
            }

            # 중복 체크
            if edge not in edges:
                edges.append(edge)

# 엣지 데이터 JSON 파일로 저장
with open('edges_0925.json', 'w', encoding='utf-8') as json_file:
    json.dump(edges, json_file, ensure_ascii=False, indent=4)


In [ ]:
# 1단계
import json

# 노드 데이터 불러오기
with open('nodes.json', 'r', encoding='utf-8') as json_file:
    nodes = json.load(json_file)

# 노드 ID 목록 생성 (본인_Id가 노드의 ID로 가정)
valid_ids = {node['본인_id'] for node in nodes}

# 엣지 데이터를 저장할 빈 리스트 생성
edges = []

# ID와 이름을 가져오는 함수
def get_id_and_name_from_node(node, category):
    id_keys = {
        '본인': '본인_id',
        '본인_부': '본인_부_id',
        '본인_조부': '본인_조부_id',
        '배우자': '배우자_id',
        '배우자_부': '배우자_부_id'
    }
    name_keys = {
        '본인': '본인_명',
        '본인_부': '본인_부_이름',
        '본인_조부': '본인_조부_이름',
        '배우자': '배우자_성명',
        '배우자_부': '배우자_부_이름'
    }
    return node[id_keys[category]], node[name_keys[category]]

def edge_exists(edges, source, target):
    return any(edge["source"] == source and edge["target"] == target for edge in edges)


# 각 노드에 대해 참조 관계 확인 후 엣지 생성
for node in nodes:
    # 1. 본인과 본인_부 연결
    source_id, source_name = get_id_and_name_from_node(node, '본인')
    target_id, target_name = get_id_and_name_from_node(node, '본인_부')
    if source_id in valid_ids and target_id:
        if len(source_id) > len(target_id):
            source_id, target_id = target_id, source_id
            source_name, target_name = target_name, source_name

        if not edge_exists(edges, source_id, target_id):
            edges.append({
                "source": source_id,
                "target": target_id,
                "source_name": source_name,
                "target_name": target_name,
                "relation": "본인 - 본인_부",
                "변수번호": node["변수번호"]
            })

# 엣지 데이터 JSON 파일로 저장
with open('edges_1004_1.json', 'w', encoding='utf-8') as json_file:
    json.dump(edges, json_file, ensure_ascii=False, indent=4)

In [ ]:
# 2단계
import json

# 노드 데이터 불러오기
with open('nodes.json', 'r', encoding='utf-8') as json_file:
    nodes = json.load(json_file)

# 노드 ID 목록 생성 (본인_Id가 노드의 ID로 가정)
valid_ids = {node['본인_id'] for node in nodes}

# 엣지 데이터를 저장할 빈 리스트 생성
edges = []

# ID와 이름을 가져오는 함수
def get_id_and_name_from_node(node, category):
    id_keys = {
        '본인': '본인_id',
        '본인_부': '본인_부_id',
        '본인_조부': '본인_조부_id',
        '배우자': '배우자_id',
        #'배우자_부': '배우자_부_id'
    }
    name_keys = {
        '본인': '본인_명',
        '본인_부': '본인_부_이름',
        '본인_조부': '본인_조부_이름',
        '배우자': '배우자_성명',
        #'배우자_부': '배우자_부_이름'
    }
    return node[id_keys[category]], node[name_keys[category]]

def edge_exists(edges, source, target):
    return any(edge["source"] == source and edge["target"] == target for edge in edges)


# 각 노드에 대해 참조 관계 확인 후 엣지 생성
for node in nodes:
    # 2. 본인_부와 본인_조부 연결
    source_id, source_name = get_id_and_name_from_node(node, '본인_부')
    target_id, target_name = get_id_and_name_from_node(node, '본인_조부')
    if source_id in valid_ids and target_id:
        if len(source_id) > len(target_id):
            source_id, target_id = target_id, source_id
            source_name, target_name = target_name, source_name

        if not edge_exists(edges, source_id, target_id):
            edges.append({
                "source": source_id,
                "target": target_id,
                "source_name": source_name,
                "target_name": target_name,
                "relation": "본인_부 - 본인_조부",
                "변수번호": node["변수번호"]
            })

# 엣지 데이터 JSON 파일로 저장
with open('edges_1004_2.json', 'w', encoding='utf-8') as json_file:
    json.dump(edges, json_file, ensure_ascii=False, indent=4)

In [ ]:
# 3단계
import json

# 노드 데이터 불러오기
with open('nodes.json', 'r', encoding='utf-8') as json_file:
    nodes = json.load(json_file)

# 노드 ID 목록 생성 (본인_Id가 노드의 ID로 가정)
valid_ids = {node['본인_id'] for node in nodes}

# 엣지 데이터를 저장할 빈 리스트 생성
edges = []

# ID와 이름을 가져오는 함수
def get_id_and_name_from_node(node, category):
    id_keys = {
        '본인': '본인_id',
        '본인_부': '본인_부_id',
        '본인_조부': '본인_조부_id',
        '배우자': '배우자_id',
        #'배우자_부': '배우자_부_id'
    }
    name_keys = {
        '본인': '본인_명',
        '본인_부': '본인_부_이름',
        '본인_조부': '본인_조부_이름',
        '배우자': '배우자_성명',
        #'배우자_부': '배우자_부_이름'
    }
    return node[id_keys[category]], node[name_keys[category]]

def edge_exists(edges, source, target):
    return any(edge["source"] == source and edge["target"] == target for edge in edges)

# 각 노드에 대해 참조 관계 확인 후 엣지 생성
for node in nodes:
    # 3. 본인과 본인_부 연결
    source_id, source_name = get_id_and_name_from_node(node, '본인')
    target_id, target_name = get_id_and_name_from_node(node, '배우자')
    if source_id in valid_ids and target_id:
        if len(source_id) > len(target_id):
            source_id, target_id = target_id, source_id
            source_name, target_name = target_name, source_name

        if not edge_exists(edges, source_id, target_id):
            edges.append({
                "source": source_id,
                "target": target_id,
                "source_name": source_name,
                "target_name": target_name,
                "relation": "본인 - 배우자",
                "변수번호": node["변수번호"]
            })

# 엣지 데이터 JSON 파일로 저장
with open('edges_1004_3.json', 'w', encoding='utf-8') as json_file:
    json.dump(edges, json_file, ensure_ascii=False, indent=4)

In [ ]:
import json

# 노드 데이터 불러오기
with open('nodes.json', 'r', encoding='utf-8') as json_file:
    nodes = json.load(json_file)

# 노드 ID 목록 생성 (본인_Id만을 기준으로 노드를 생성한다고 가정)
valid_ids = {node['본인_id'] for node in nodes}

# 엣지 데이터를 저장할 빈 리스트 생성
edges = []

# ID와 이름을 가져오는 함수
def get_id_and_name_from_node(node, category):
    id_keys = {
        '본인': '본인_id',
        '본인_부': '본인_부_id',
    }
    name_keys = {
        '본인': '본인_명',
        '본인_부': '본인_부_이름',
    }
    return node[id_keys[category]], node[name_keys[category]]

def edge_exists(edges, source, target):
    return any(edge["source"] == source and edge["target"] == target for edge in edges)

# 각 노드에 대해 참조 관계 확인 후 엣지 생성
for node in nodes:
    source_id, source_name = get_id_and_name_from_node(node, '본인')
    target_id, target_name = get_id_and_name_from_node(node, '본인_부')

    # 본인과 배우자의 ID가 모두 valid_ids(본인_id들의 집합)에 존재하면 엣지를 생성
    if source_id in valid_ids and target_id in valid_ids:
        if not edge_exists(edges, source_id, target_id):
            edges.append({
                "source": source_id,
                "target": target_id,
                "source_name": source_name,
                "target_name": target_name,
                "relation": "본인 - 본인_부",
                "변수번호": node["변수번호"]
            })

# 엣지 데이터 JSON 파일로 저장
with open('edges_04_2.json', 'w', encoding='utf-8') as json_file:
    json.dump(edges, json_file, ensure_ascii=False, indent=4)


In [ ]:
# 얘가 문제되는듯..
import json

# 노드 데이터 불러오기
with open('nodes.json', 'r', encoding='utf-8') as json_file:
    nodes = json.load(json_file)

# 노드 ID 목록 생성 (본인_Id만을 기준으로 노드를 생성한다고 가정)
valid_ids = {node['본인_id'] for node in nodes}

# 엣지 데이터를 저장할 빈 리스트 생성
edges = []

# ID와 이름을 가져오는 함수
def get_id_and_name_from_node(node, category):
    id_keys = {
        '본인_부': '본인_부_id',
        '본인_조부': '본인_조부_id',
    }
    name_keys = {
        '본인_부': '본인_부_이름',
        '본인_조부': '본인_조부_이름',
    }
    return node[id_keys[category]], node[name_keys[category]]

def edge_exists(edges, source, target):
    return any(edge["source"] == source and edge["target"] == target for edge in edges)

# 각 노드에 대해 참조 관계 확인 후 엣지 생성
for node in nodes:
    source_id, source_name = get_id_and_name_from_node(node, '본인_부')
    target_id, target_name = get_id_and_name_from_node(node, '본인_조부')

    # 본인과 배우자의 ID가 모두 valid_ids(본인_id들의 집합)에 존재하면 엣지를 생성
    if source_id in valid_ids and target_id in valid_ids:
        if not edge_exists(edges, source_id, target_id):
            edges.append({
                "source": source_id,
                "target": target_id,
                "source_name": source_name,
                "target_name": target_name,
                "relation": "본인_부 - 본인_조부",
                "변수번호": node["변수번호"]
            })

# 엣지 데이터 JSON 파일로 저장
with open('edges_04_1.json', 'w', encoding='utf-8') as json_file:
    json.dump(edges, json_file, ensure_ascii=False, indent=4)


In [ ]:
import json

# 노드 데이터 불러오기
with open('nodes.json', 'r', encoding='utf-8') as json_file:
    nodes = json.load(json_file)

# 노드 ID 목록 생성 (본인_Id만을 기준으로 노드를 생성한다고 가정)
valid_ids = {node['본인_id'] for node in nodes}
valid_ids.update(node['배우자_id'] for node in nodes if node['배우자_id'] is not None)

# 엣지 데이터를 저장할 빈 리스트 생성
edges = []

# ID와 이름을 가져오는 함수
def get_id_and_name_from_node(node, category):
    id_keys = {
        '본인': '본인_id',
        '배우자': '배우자_id',
    }
    name_keys = {
        '본인': '본인_명',
        '배우자': '배우자_성명',
    }
    return node[id_keys[category]], node[name_keys[category]]

def edge_exists(edges, source, target):
    return any(edge["source"] == source and edge["target"] == target for edge in edges)

# 각 노드에 대해 참조 관계 확인 후 엣지 생성
for node in nodes:
    source_id, source_name = get_id_and_name_from_node(node, '본인')
    target_id, target_name = get_id_and_name_from_node(node, '배우자')

    # 본인과 배우자의 ID가 모두 valid_ids(본인_id들의 집합)에 존재하면 엣지를 생성
    if source_id in valid_ids and target_id in valid_ids:
        if not edge_exists(edges, source_id, target_id):
            edges.append({
                "source": source_id,
                "target": target_id,
                "source_name": source_name,
                "target_name": target_name,
                "relation": "본인 - 배우자",
                "변수번호": node["변수번호"]
            })

# 엣지 데이터 JSON 파일로 저장
with open('edges_04_4.json', 'w', encoding='utf-8') as json_file:
    json.dump(edges, json_file, ensure_ascii=False, indent=4)


In [ ]:
import json
import networkx as nx

# 노드와 엣지 데이터 불러오기
with open('nodes.json', 'r', encoding='utf-8') as json_file:
    nodes = json.load(json_file)

with open('edgesss.json', 'r', encoding='utf-8') as json_file:
    edges = json.load(json_file)

# 그래프 생성
G = nx.Graph()

# 노드 추가
for node in nodes:
    G.add_node(node['본인_id'], name=node['본인_명'])

# 엣지 추가
for edge in edges:
    G.add_edge(edge['source'], edge['target'])

# 중심성 메트릭 계산
betweenness = nx.betweenness_centrality(G)
pagerank = nx.pagerank(G)
closeness = nx.closeness_centrality(G)
degree = dict(G.degree())

# 모든 중심성 메트릭을 하나의 딕셔너리에 담기
centrality_metrics = {
    'betweenness': betweenness,
    'pagerank': pagerank,
    'closeness': closeness,
    'degree': degree
}

# 중심성 메트릭을 JSON 파일로 저장
with open('metrics_1004.json', 'w', encoding='utf-8') as json_file:
    json.dump(centrality_metrics, json_file, ensure_ascii=False, indent=4)
